<a href="https://colab.research.google.com/github/venkat52/my_colab_notes/blob/main/Module3_Lab3_fmml20210326.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad**
### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification
#### Module Coordinator: Sahil Manoj Bhatt

NOTE: YOU ONLY NEED TO MAKE CHANGES/WRITE CODE IN CELLS THAT SPECIFICALLY MENTION TASK-1, TASK-2, etc.

WRITE ANY OBSERVATION(S), IF REQUIRED BY THE TASK, IN A SEPARATE CELL AT THE BOTTOM OF THE NOTEBOOK.  

---

In [ ]:
# NLTK (or Natural Language Tool Kit) is a commonly used library for processing text.
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



In [ ]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [ ]:
# cleanText('jdaj44o2avmsv444',True,False)
isinstance('2.00',float)
'sdsg'.decode()

AttributeError: ignored

In [ ]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

Troubling
troubl
trouble


### BAG OF WORDS 

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document. 
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [ ]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


### TF-IDF 
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*. 




In [ ]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

### UNDERSTANDING THE DATA : A REVIEWS DATASET

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [ ]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews.csv


In [ ]:
import pandas as pd
df = pd.read_csv('reviews.csv',sep='\t')
df.head(20)

,sentence,sentiment
0,Not sure who was more lost - the flat characte...,0
1,Attempting artiness with black & white and cle...,0
2,Very little music or anything to speak of.,0
3,The best scene in the movie was when Gerardo i...,1
4,"The rest of the movie lacks art, charm, meanin...",0
5,Wasted two hours.,0
6,Saw the movie today and thought it was a good ...,1
7,A bit predictable.,0
8,Loved the casting of Jimmy Buffet as the scien...,1
9,And those baby owls were adorable.,1


### KNN MODEL

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

weight = ['uniform','distance']
dist_metric = ['minkowski','hamming','manhattan','cosine','chebyshev','jaccard','euclidean']


def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""
    w=[]
    for i in weight:
      for j in dist_metric:


        training_data = pd.read_csv('reviews.csv',sep='\t')
        X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
        X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
        # print(X_train)
        knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights=i, algorithm='auto', leaf_size=30, p=2, metric=j, metric_params=None, n_jobs=1)

        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, predicted)
        print('KNN with BOW accuracy with '+str(i)+' and distace_metric='+str(j)+ ' = ' + str(acc * 100) + '%')

        scores = cross_val_score(knn, X_train, y_train, cv=3)
        print("Cross Validation Accuracy with "+str(i)+" and distace_metric="+str(j)+": %0.2f" % (scores.mean()))
        print(scores)
        print('\n')
        w.append((predicted, y_test))
    return w[0]


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""
    w=[]
    for i in weight:
      for j in dist_metric:
        training_data = pd.read_csv('reviews.csv',sep='\t')
        X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                            test_size=0.2, random_state=5)
        X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
        # print(X_train)
        knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights=i, algorithm='brute', leaf_size=30, p=2,
                                            metric=j, metric_params=None, n_jobs=1)

        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, predicted)
        print('KNN with BOW accuracy with '+str(i)+' and distace_metric='+str(j)+ ' = ' + str(acc * 100) + '%')

        scores = cross_val_score(knn, X_train, y_train, cv=3)
        print("Cross Validation Accuracy with "+str(i)+" and distace_metric="+str(j)+": %0.2f" % (scores.mean()))
        print(scores)
        print()
        w.append((predicted, y_test))
    return w[0]

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [ ]:
## KNN accuracy after using BoW
import pandas as pd
predicted, y_test = bow_knn()

KNN with BOW accuracy with uniform and distace_metric=minkowski = 66.5%
Cross Validation Accuracy with uniform and distace_metric=minkowski: 0.61
[0.61423221 0.59398496 0.61278195]


KNN with BOW accuracy with uniform and distace_metric=hamming = 70.0%
Cross Validation Accuracy with uniform and distace_metric=hamming: 0.61
[0.62172285 0.60150376 0.60526316]


KNN with BOW accuracy with uniform and distace_metric=manhattan = 68.5%
Cross Validation Accuracy with uniform and distace_metric=manhattan: 0.64
[0.65543071 0.64285714 0.63157895]


KNN with BOW accuracy with uniform and distace_metric=cosine = 71.0%
Cross Validation Accuracy with uniform and distace_metric=cosine: 0.71
[0.68913858 0.72180451 0.70676692]


KNN with BOW accuracy with uniform and distace_metric=chebyshev = 48.5%
Cross Validation Accuracy with uniform and distace_metric=chebyshev: 0.49
[0.4494382  0.4924812  0.54135338]




/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


KNN with BOW accuracy with uniform and distace_metric=jaccard = 74.0%


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Cross Validation Accuracy with uniform and distace_metric=jaccard: 0.70
[0.72284644 0.69924812 0.69172932]


KNN with BOW accuracy with uniform and distace_metric=euclidean = 66.5%
Cross Validation Accuracy with uniform and distace_metric=euclidean: 0.61
[0.61423221 0.59398496 0.61278195]


KNN with BOW accuracy with distance and distace_metric=minkowski = 66.5%
Cross Validation Accuracy with distance and distace_metric=minkowski: 0.61
[0.61423221 0.59774436 0.62030075]


KNN with BOW accuracy with distance and distace_metric=hamming = 70.0%
Cross Validation Accuracy with distance and distace_metric=hamming: 0.61
[0.61797753 0.60526316 0.59774436]


KNN with BOW accuracy with distance and distace_metric=manhattan = 68.5%
Cross Validation Accuracy with distance and distace_metric=manhattan: 0.65
[0.67041199 0.64661654 0.64285714]


KNN with BOW accuracy with distance and distace_metric=cosine = 71.0%
Cross Validation Accuracy with distance and distace_metric=cosine: 0.71
[0.69662921 0.7

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


KNN with BOW accuracy with distance and distace_metric=jaccard = 74.0%


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Cross Validation Accuracy with distance and distace_metric=jaccard: 0.71
[0.73033708 0.69924812 0.69172932]


KNN with BOW accuracy with distance and distace_metric=euclidean = 66.5%
Cross Validation Accuracy with distance and distace_metric=euclidean: 0.61
[0.61423221 0.59774436 0.62030075]




In [ ]:
## KNN accuracy after using TFIDF
predicted, y_test = tfidf_knn()

KNN with BOW accuracy with uniform and distace_metric=minkowski = 72.5%
Cross Validation Accuracy with uniform and distace_metric=minkowski: 0.69
[0.73782772 0.66917293 0.65413534]

KNN with BOW accuracy with uniform and distace_metric=hamming = 65.5%
Cross Validation Accuracy with uniform and distace_metric=hamming: 0.55
[0.55430712 0.57894737 0.52255639]

KNN with BOW accuracy with uniform and distace_metric=manhattan = 67.5%
Cross Validation Accuracy with uniform and distace_metric=manhattan: 0.63
[0.66292135 0.62030075 0.59774436]

KNN with BOW accuracy with uniform and distace_metric=cosine = 76.0%
Cross Validation Accuracy with uniform and distace_metric=cosine: 0.73
[0.73033708 0.7518797  0.70676692]

KNN with BOW accuracy with uniform and distace_metric=chebyshev = 62.5%
Cross Validation Accuracy with uniform and distace_metric=chebyshev: 0.56
[0.54681648 0.59022556 0.55263158]



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


KNN with BOW accuracy with uniform and distace_metric=jaccard = 74.0%


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Cross Validation Accuracy with uniform and distace_metric=jaccard: 0.70
[0.72284644 0.69924812 0.69172932]

KNN with BOW accuracy with uniform and distace_metric=euclidean = 72.5%
Cross Validation Accuracy with uniform and distace_metric=euclidean: 0.69
[0.73782772 0.66917293 0.65413534]

KNN with BOW accuracy with distance and distace_metric=minkowski = 72.5%
Cross Validation Accuracy with distance and distace_metric=minkowski: 0.69
[0.73782772 0.67293233 0.65413534]

KNN with BOW accuracy with distance and distace_metric=hamming = 66.0%
Cross Validation Accuracy with distance and distace_metric=hamming: 0.55
[0.55430712 0.58646617 0.52255639]

KNN with BOW accuracy with distance and distace_metric=manhattan = 67.5%
Cross Validation Accuracy with distance and distace_metric=manhattan: 0.62
[0.66666667 0.61278195 0.59398496]

KNN with BOW accuracy with distance and distace_metric=cosine = 76.0%
Cross Validation Accuracy with distance and distace_metric=cosine: 0.73
[0.73033708 0.748120

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


KNN with BOW accuracy with distance and distace_metric=jaccard = 74.0%


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/pairwise.py:1870: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Cross Validation Accuracy with distance and distace_metric=jaccard: 0.71
[0.73033708 0.69924812 0.69172932]

KNN with BOW accuracy with distance and distace_metric=euclidean = 72.5%
Cross Validation Accuracy with distance and distace_metric=euclidean: 0.69
[0.73782772 0.67293233 0.65413534]



### SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [ ]:
# Upload the spam text data CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving spam_text_data.csv to spam_text_data.csv


In [ ]:
import pandas as pd
df = pd.read_csv('spam_text_data.csv', error_bad_lines=False)
df.head(20)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [ ]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [ ]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
len(df)

5572

In [ ]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.
weight = ['uniform','distance']
dist_metric = ['minkowski','hamming','manhattan','cosine','chebyshev','jaccard','euclidean']

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""
    
    w=[]
    for i in weight:
      for j in dist_metric:
        training_data = pd.read_csv('spam_text_data.csv')
        training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
        X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
        X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
        knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, predicted)
        print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

        scores = cross_val_score(knn, X_train, y_train, cv=3)
        print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
        print(scores)
        print('\n')
        w.append((predicted, y_test))
    return w[0]


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""
    w=[]
    for i in weight:
      for j in dist_metric:
        training_data = pd.read_csv('spam_text_data.csv')
        training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
        X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
        X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
        knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

        knn.fit(X_train, y_train)
        predicted = knn.predict(X_test)
        acc = metrics.accuracy_score(y_test, predicted)
        print('KNN with BOW accuracy with '+str(i)+' and distace_metric='+str(j)+ ' = ' + str(acc * 100) + '%')

        scores = cross_val_score(knn, X_train, y_train, cv=3)
        print("Cross Validation Accuracy with "+str(i)+" and distace_metric="+str(j)+": %0.2f" % (scores.mean()))
        print(scores)
        w.append((predicted, y_test))
        print()
    return w[0]

In [ ]:
# This cell may take some time to run
predicted, y_test = bow_knn()

KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]


KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.9124

In [ ]:
# This cell may take some time to run
predicted, y_test = tfidf_knn()

KNN with BOW accuracy with uniform and distace_metric=minkowski = 98.56502242152466%
Cross Validation Accuracy with uniform and distace_metric=minkowski: 0.97
[0.96837147 0.96769852 0.96363636]

KNN with BOW accuracy with uniform and distace_metric=hamming = 98.56502242152466%
Cross Validation Accuracy with uniform and distace_metric=hamming: 0.97
[0.96837147 0.96769852 0.96363636]

KNN with BOW accuracy with uniform and distace_metric=manhattan = 98.56502242152466%
Cross Validation Accuracy with uniform and distace_metric=manhattan: 0.97
[0.96837147 0.96769852 0.96363636]

KNN with BOW accuracy with uniform and distace_metric=cosine = 98.56502242152466%
Cross Validation Accuracy with uniform and distace_metric=cosine: 0.97
[0.96837147 0.96769852 0.96363636]

KNN with BOW accuracy with uniform and distace_metric=chebyshev = 98.56502242152466%
Cross Validation Accuracy with uniform and distace_metric=chebyshev: 0.97
[0.96837147 0.96769852 0.96363636]

KNN with BOW accuracy with uniform 

### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html


### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
2. Can you think of techniques that are better than both BoW and TF-IDF ?
3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

**ANSWER TO Q)1:**In tf-idf it will always try to give importance to uniquness or rare words where as Bag-of-words will just return a vector corresponding to word as their count.so this information which returned by bag-of-words will not be so use to find distnict when compare to TF-IDF. So we generally prefer TF-IDF over Bag-of-words

**ANSWER TO Q)2:**word-embedding,word2Vector,universal-encoding are some of other techinque which is better than both BoW and TF-IDF

**ANSWER TO Q)3:**Both stemming and Lemmatization is to reduce a word to it's basic form .

Advantages of Stemming


1.   It is faster compare to lemming


Disadvantage of stemming


1.   sometimes the lemming will return words which donot have meaning

Advantages of lemming



1.   It always return the words which have meaning

Disadvantage of lemming



1.   It takes more time 







